In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor

In [2]:
# Load client data
client = "elgiganten"
client_df = pd.read_excel(f"./{client}_input_data.xlsx", converters={"Imps": int, "Placement Id": int})
client_df["Viewability Rate %"] = client_df["Viewability Rate %"].str.replace(",", ".").astype(float)
client_placement_ids = list(client_df["Placement Id"].unique())

In [3]:
# Fetch impressions using PyAthena library
cursor = connect(s3_staging_dir="s3://aws-athena-query-results-094611745175-eu-west-1/",
                 region_name="eu-west-1", profile_name="atexprodadminsso", cursor_class=PandasCursor).cursor()

impressions_df = cursor.execute(f'''
select 
    "impression_model"."id", 
    "hostname", 
    "placement_ids", 
    "placement_ids_chosen", 
    "total_fixation_duration", 
    "channel", 
    "ad_technical_format", 
    "is_fixated", 
    "gaze_valid", 
    "is_iab_inview", 
    "exist_viewable_1_s_threshold_50", 
    "exist_viewable_2_s_threshold_50", 
    "impression_model"."part_month", 
    CONCAT(cast("ad_width_chosen" as VARCHAR), 'x', cast("ad_width_chosen" as VARCHAR)) as size,
    "brand_batch"."chosen_brand" as "chosen_brand"
from "prod_attentionpanel_com_eu_west_1"."impression_model"
join "prod_attentionpanel_com_eu_west_1"."brand_batch" on impression_model.id = brand_batch.id
where impression_model.part_year = '2023'
''').as_pandas()

In [4]:
def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    try:
        ret = placement_id['tag_id'][0]
    except:
        return
    ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_16041/3375560408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [5]:
# Get local predictions and store them in a map
def _get_pred_map():
    pred_map = dict()
    _year = 2023
    _range1 = 1
    _range2 = 10
    for i in range(_range1, _range2):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../../predictions/{_year}/{i}'):
            for h in os.listdir(f'../../predictions/{_year}/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../../predictions/{_year}/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [34]:
# Filter impressions based on tier
### TIER 1 ###
# This is the most accurate data we have. This is "campaign data", meaning correct brand and timeframe
### TIER 2 ###
# We might not have enough campaign data (tier 1). Tier 2 data is brand data, for a longer period (roughly 6 months)
### TIER 3 ###
# This is the least accurate data for a campaign. It ignore brand/timeframe and only checks Placement ID. This is mostly used as a benchmark
tier = 3
filtered_impressions = impressions_df[impressions_df["pid"].isin(client_placement_ids)]
if tier == 1:
    filtered_impressions = filtered_impressions[filtered_impressions["part_month"] == "09"]
if tier < 3:
    filtered_impressions = filtered_impressions[filtered_impressions["chosen_brand"] == client.lower()]

In [35]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

/tmp/ipykernel_16041/1232414527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)


In [36]:
filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

/tmp/ipykernel_16041/2412417818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(


In [37]:
filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")
# Normalise outliers for fixation duration (30 seconds)
filtered_impressions.loc[filtered_impressions["total_fixation_duration"] >= 30000, "total_fixation_duration"] = 30000

/tmp/ipykernel_16041/3020262113.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")


In [38]:
# Predictions do not include fixation duration. For predicted fixations, use the median value as the total_fixation_duration

# Calculate the median fixation duration for each hostname
median_fixation_durations = filtered_impressions.loc[filtered_impressions['is_fixated'] == True].groupby('hostname')['total_fixation_duration'].median()

# Update the total_fixation_duration based on the calculated medians
filtered_impressions['total_fixation_duration'] = np.where(
    (filtered_impressions['total_fixation_duration'] == 0) &
    (filtered_impressions['final_fixation'] == True) &
    (filtered_impressions['hostname'].isin(median_fixation_durations.index)),
    filtered_impressions['hostname'].map(median_fixation_durations),
    filtered_impressions['total_fixation_duration']
)

# If still 0, Calculate the median fixation duration for each brand/size
median_fixation_durations = filtered_impressions.loc[filtered_impressions['is_fixated'] == True].groupby(["chosen_brand", "size"])['total_fixation_duration'].median()

# Update the total_fixation_duration based on the calculated medians
filtered_impressions['total_fixation_duration'] = np.where(
    (filtered_impressions['total_fixation_duration'] == 0) &
    (filtered_impressions['final_fixation'] == True) &
    (filtered_impressions['chosen_brand'].isin(median_fixation_durations.index)) & 
    (filtered_impressions['size'].isin(median_fixation_durations.index)),
    filtered_impressions['chosen_brand'].map(median_fixation_durations),
    filtered_impressions['total_fixation_duration']
)

# If still 0, Calculate the median fixation duration for each brand
median_fixation_durations = filtered_impressions.loc[filtered_impressions['is_fixated'] == True].groupby("chosen_brand")['total_fixation_duration'].median()

# Update the total_fixation_duration based on the calculated medians
filtered_impressions['total_fixation_duration'] = np.where(
    (filtered_impressions['total_fixation_duration'] == 0) &
    (filtered_impressions['final_fixation'] == True) &
    (filtered_impressions['chosen_brand'].isin(median_fixation_durations.index)),
    filtered_impressions['chosen_brand'].map(median_fixation_durations),
    filtered_impressions['total_fixation_duration']
)

# If still 0, Calculate the median fixation duration for each size
median_fixation_durations = filtered_impressions.loc[filtered_impressions['is_fixated'] == True].groupby("size")['total_fixation_duration'].median()

# Update the total_fixation_duration based on the calculated medians
filtered_impressions['total_fixation_duration'] = np.where(
    (filtered_impressions['total_fixation_duration'] == 0) &
    (filtered_impressions['final_fixation'] == True) &
    (filtered_impressions['size'].isin(median_fixation_durations.index)),
    filtered_impressions['size'].map(median_fixation_durations),
    filtered_impressions['total_fixation_duration']
)

/tmp/ipykernel_16041/3042594107.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions['total_fixation_duration'] = np.where(
/tmp/ipykernel_16041/3042594107.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_impressions['total_fixation_duration'] = np.where(
/tmp/ipykernel_16041/3042594107.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [39]:
grouped_df = filtered_impressions.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),
            "fixations": x.loc[x["final_fixation"], "id"].count(),
            "inview": x.loc[x["true_is_iab_inview"], "id"].count(),
            "total_fixation_duration": x["total_fixation_duration"].sum(),
            "average_fixation_duration": x["total_fixation_duration"].mean()
        }
    )
)
grouped_df = grouped_df.reset_index()

In [40]:
grouped_client = client_df.groupby(["Placement Id"]).apply(
    lambda x: pd.Series(
        {
            "Imps": x["Imps"].sum(),
            "Viewability Rate %": np.average(x["Viewability Rate %"], weights=x["Imps"])
        }
    )
)
grouped_client = grouped_client.reset_index()

In [41]:
merged_df = grouped_df.merge(grouped_client, how="inner", left_on="pid", right_on="Placement Id")
merged_df = merged_df[["hostname", "Placement Id", "impressions", "fixations", "inview", "Imps", "Viewability Rate %", "average_fixation_duration", "total_fixation_duration"]]
merged_df["fixation_ratio"] = round(merged_df["fixations"] / merged_df["impressions"], 2)
merged_df["client_fixations_per_placement"] = (merged_df["fixation_ratio"] * merged_df["Imps"]).fillna(0).replace([np.inf, -np.inf], 0)

In [62]:
grouped_df[g["hostname"] == "tramsfrans.aftonbladet.se"]

,hostname,Placement Id,impressions,fixations,inview,Imps,Viewability Rate %,average_fixation_duration,total_fixation_duration,fixation_ratio,client_fixations_per_placement
10,tramsfrans.aftonbladet.se,19499417,1.0,0.0,1.0,2813.0,0.900916,0.0,0.0,0.0,0.0
70,tramsfrans.aftonbladet.se,19500003,1.0,0.0,1.0,28930.0,0.617546,0.0,0.0,0.0,0.0


In [63]:
merged_df[merged_df["hostname"] == "skonhetsredaktorerna.se"]

,hostname,Placement Id,impressions,fixations,inview,Imps,Viewability Rate %,average_fixation_duration,total_fixation_duration,fixation_ratio,client_fixations_per_placement
9,skonhetsredaktorerna.se,19499417,1.0,1.0,1.0,2813.0,0.900916,1718.0,1718.0,1.0,2813.0
69,skonhetsredaktorerna.se,19500003,1.0,1.0,1.0,28930.0,0.617546,1821.0,1821.0,1.0,28930.0


In [43]:
final_group = merged_df.groupby(["hostname"]).apply(
     lambda x: pd.Series(
         {
             "tobii_imps": x["impressions"].sum(),
             "client_imps": x["Imps"].sum(),
             "tobii_fixations": x["fixations"].sum(),
             "tobii_inview": x["inview"].sum(),
             "client_fixations_per_placement": x["client_fixations_per_placement"].sum(),
             "client_viewability_rate": np.average(x["Viewability Rate %"], weights=x["Imps"]),
             "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),
             "total_fixation_duration": x["total_fixation_duration"].sum()
         }
     )
)
final_group = final_group.reset_index()
final_group["tobii_imps"] = final_group["tobii_imps"].astype("Int64")
final_group["client_imps"] = final_group["client_imps"].astype("Int64")
final_group["tobii_fixations"] = final_group["tobii_fixations"].astype("Int64")
final_group["tobii_inview"] = final_group["tobii_inview"].astype("Int64")
final_group["tobii_inview/impression_ratio"] = round(final_group["tobii_inview"] / final_group["tobii_imps"] * 100, 2)
final_group["tobii_fixation/impression_ratio"] = round(final_group["tobii_fixations"] / final_group["tobii_imps"] * 100, 2)

/tmp/ipykernel_16041/3953223301.py:10: RuntimeWarning: invalid value encountered in scalar divide
  "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),


In [44]:
final_group = final_group.fillna(0).replace([np.inf, -np.inf], 0)
final_group

,hostname,tobii_imps,client_imps,tobii_fixations,tobii_inview,client_fixations_per_placement,client_viewability_rate,average_fixation_duration,total_fixation_duration,tobii_inview/impression_ratio,tobii_fixation/impression_ratio
0,aftonbladet.se,145754,2348672,36030,86496,666735.39,0.688627,3142.0,113196710.0,59.34,24.72
1,bloggar.aftonbladet.se,375,120076,198,199,60510.98,0.633919,3929.0,777946.0,53.07,52.8
2,klart.se,1328,3988,402,738,1496.87,0.631551,1902.0,764510.0,55.57,30.27
3,malservice.aftonbladet.se,67,4,13,20,0.76,0.000000,654.0,8496.5,29.85,19.4
4,skonhetsredaktorerna.se,2,31743,2,2,31743.00,0.642658,1770.0,3539.0,100.0,100.0
5,svd.se,12234,28007,5837,7883,10237.78,0.734596,8452.0,49332239.0,64.44,47.71
6,tramsfrans.aftonbladet.se,2,31743,0,2,0.00,0.642658,0.0,0.0,100.0,0.0
7,tv.nu,853,30742,228,504,7597.95,0.655062,2273.0,518162.0,59.09,26.73


In [45]:
final_group["client_inview"] = round(final_group["client_imps"] * final_group["client_viewability_rate"], 0).astype("Int64")
final_group["client_fixations"] = round(final_group["client_fixations_per_placement"], 0).astype("Int64")
final_group["client_fix/inview_ratio"] = round(final_group["client_fixations"] / final_group["client_inview"] * 100, 2)
final_group["client_total_fixation_duration"] = final_group["client_fixations"] * final_group["average_fixation_duration"]

def get_sample_size(num_impressions):
    if num_impressions <= 99:
        return "low"
    elif num_impressions >= 100 and num_impressions <= 199:
        return "medium"
    else:
        return "high"

final_group["sample_size"] = final_group["tobii_imps"].apply(lambda x: get_sample_size(x))

In [46]:
final_group

,hostname,tobii_imps,client_imps,tobii_fixations,tobii_inview,client_fixations_per_placement,client_viewability_rate,average_fixation_duration,total_fixation_duration,tobii_inview/impression_ratio,tobii_fixation/impression_ratio,client_inview,client_fixations,client_fix/inview_ratio,client_total_fixation_duration,sample_size
0,aftonbladet.se,145754,2348672,36030,86496,666735.39,0.688627,3142.0,113196710.0,59.34,24.72,1617359,666735,41.22,2094881370.0,high
1,bloggar.aftonbladet.se,375,120076,198,199,60510.98,0.633919,3929.0,777946.0,53.07,52.8,76118,60511,79.5,237747719.0,high
2,klart.se,1328,3988,402,738,1496.87,0.631551,1902.0,764510.0,55.57,30.27,2519,1497,59.43,2847294.0,high
3,malservice.aftonbladet.se,67,4,13,20,0.76,0.000000,654.0,8496.5,29.85,19.4,0,1,inf,654.0,low
4,skonhetsredaktorerna.se,2,31743,2,2,31743.00,0.642658,1770.0,3539.0,100.0,100.0,20400,31743,155.6,56185110.0,low
5,svd.se,12234,28007,5837,7883,10237.78,0.734596,8452.0,49332239.0,64.44,47.71,20574,10238,49.76,86531576.0,high
6,tramsfrans.aftonbladet.se,2,31743,0,2,0.00,0.642658,0.0,0.0,100.0,0.0,20400,0,0.0,0.0,low
7,tv.nu,853,30742,228,504,7597.95,0.655062,2273.0,518162.0,59.09,26.73,20138,7598,37.73,17270254.0,high


In [48]:
result_df = final_group[["hostname", "tobii_imps", "client_imps", "tobii_fixations", "client_fixations", "tobii_inview", "client_inview", "client_fix/inview_ratio", "tobii_inview/impression_ratio", "client_viewability_rate", "tobii_fixation/impression_ratio", "average_fixation_duration", "total_fixation_duration", "client_total_fixation_duration", "sample_size"]]
result_df
result_df.to_excel(f"./{client.lower()}_tier_{tier}_example_report.xlsx", index=False)

In [49]:
# Load tiered data
df_1 = pd.read_excel(f"./{client.lower()}_tier_1_example_report.xlsx")
df_1["tier"] = 1
df_2 = pd.read_excel(f"./{client.lower()}_tier_2_example_report.xlsx")
df_2["tier"] = 2
df_3 = pd.read_excel(f"./{client.lower()}_tier_3_example_report.xlsx")
df_3["tier"] = 3
all_df = [df_1, df_2, df_3]
combined_client_df = pd.concat(all_df)
columns = list(df_1.columns)

In [50]:
def get_data_based_on_tier(client_dfs):
    new_df = pd.DataFrame(columns=columns)
    for i, df in enumerate(client_dfs):
        for index, row in df.iterrows():
            if row.tier == 1:
                if row.sample_size in ("high", "medium"):
                    new_df = new_df.append(row)
            elif row.tier == 2:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0 and row.tobii_imps >= 10:
                    if row.sample_size == "high":
                        # Degrade sample_size if lower tier
                        row.sample_size = "medium"
                        new_df = new_df.append(row)
                    elif row.sample_size == "medium":
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.25 + new_row["client_fixations"].sum() * 0.75, 0)
                        )
                        new_df = new_df.append(new_row)
                    else:
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.15 + new_row["client_fixations"].sum() * 0.85, 0)
                        )
                        new_df = new_df.append(new_row)
            elif row.tier == 3:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    row["sample_size"] = "low"
                    new_df = new_df.append(row)
    return new_df

final_client_df = get_data_based_on_tier(all_df)

/tmp/ipykernel_16041/4216252792.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
/tmp/ipykernel_16041/4216252792.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.sample_size = "medium"
/tmp/ipykernel_16041/4216252792.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["client_fixations"] = (
/tmp/ipykernel_16041/4216252792.py:27: FutureWarning: The frame.append method 

In [51]:
final_client_df

,hostname,tobii_imps,client_imps,tobii_fixations,client_fixations,tobii_inview,client_inview,client_fix/inview_ratio,tobii_inview/impression_ratio,client_viewability_rate,tobii_fixation/impression_ratio,average_fixation_duration,total_fixation_duration,client_total_fixation_duration,sample_size,tier
0,aftonbladet.se,101,1360677,41,320780,62,952920,33.66,61.39,0.700328,40.59,829,34007.0,265926620,medium,1
1,bloggar.aftonbladet.se,375,120076,198,54004.0,199,76118,79.50,53.07,0.633919,52.80,3929,777946.0,237747719,medium,3
2,klart.se,1328,3988,402,1487.0,738,2519,59.43,55.57,0.631551,30.27,1902,764510.0,2847294,medium,3
3,svd.se,335,27227,115,10917,256,20017,54.54,76.42,0.735190,34.33,3373,387877.0,36823041,medium,2
7,tv.nu,853,30742,228,8482.0,504,20138,37.73,59.09,0.655062,26.73,2273,518162.0,17270254,medium,3
3,malservice.aftonbladet.se,67,4,13,1,20,0,inf,29.85,0.000000,19.40,654,8496.5,654,low,3
4,skonhetsredaktorerna.se,2,31743,2,31743,2,20400,155.60,100.00,0.642658,100.00,1770,3539.0,56185110,low,3
6,tramsfrans.aftonbladet.se,2,31743,0,0,2,20400,0.00,100.00,0.642658,0.00,0,0.0,0,low,3


In [53]:
def do_math(row, arg1, arg2):
    try:
        return round(row[f"{arg1}"] / row[f"{arg2}"] * 100, 2)
    except ZeroDivisionError:
        return 0

final_client_df["client_Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_inview", "client_imps"), axis=1)
final_client_df["client_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_client_df["tobii_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_client_df["client_fixation/impression_ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_client_df["Total Fixation Duration"] = final_client_df["average_fixation_duration"] * final_client_df["client_fixations"]
final_client_df = final_client_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "Total Fixation Duration", "average_fixation_duration", "sample_size", "tier"]]
final_client_df = final_client_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        "client_Inview Ratio": "Inview Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "average_fixation_duration": "Average Fixation Duration",
        "sample_size": "Sample Size"
    }
)
final_client_df

,hostname,Impressions,Fixations,tobii_fixation/impression_ratio,Inviews,Inview Ratio,Fixation/Inview Ratio,Fixation/Tobii Inview Ratio,tobii_inview/impression_ratio,Total Fixation Duration,Average Fixation Duration,Sample Size,tier
0,aftonbladet.se,1360677,320780,40.59,952920,70.03,33.66,66.13,61.39,265926620,829,medium,1
1,bloggar.aftonbladet.se,120076,54004.0,52.80,76118,63.39,70.95,99.50,53.07,212181716.0,3929,medium,3
2,klart.se,3988,1487.0,30.27,2519,63.16,59.03,54.47,55.57,2828274.0,1902,medium,3
3,svd.se,27227,10917,34.33,20017,73.52,54.54,44.92,76.42,36823041,3373,medium,2
7,tv.nu,30742,8482.0,26.73,20138,65.51,42.12,45.24,59.09,19279586.0,2273,medium,3
3,malservice.aftonbladet.se,4,1,19.40,0,0.00,0.00,65.00,29.85,654,654,low,3
4,skonhetsredaktorerna.se,31743,31743,100.00,20400,64.27,155.60,100.00,100.00,56185110,1770,low,3
6,tramsfrans.aftonbladet.se,31743,0,0.00,20400,64.27,0.00,0.00,100.00,0,0,low,3


In [54]:
final_client_df = final_client_df[["hostname", "Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration", "Sample Size"]]
final_client_df = final_client_df.sort_values(by=["hostname"])

In [55]:
comments = ["Tier 1", "Tier 2", "Tier 3"]

with pd.ExcelWriter(f"./{client}_results_internal.xlsx") as writer:
    rows = 0
    spaces = 0
    for n, df in enumerate(all_df):
        pd.Series(comments[n]).to_excel(writer, sheet_name=client.title(), index=False, header=False, startrow=rows + spaces)
        df.to_excel(writer, client.title(), index=False, startrow=1 + rows + spaces)
        rows += len(df) + 2
        spaces += 2

with pd.ExcelWriter(f"./{client}_results.xlsx") as writer:
    for df in [final_client_df]:
        df.to_excel(writer, sheet_name=client.title(), index=False)